# Part IV: Machine learning

![](img/machine_learning.png)

In this section we'll learn:

- Regression models
- Classification models
- Measuring model errors
- The [scikit-learn](http://scikit-learn.org/) API

In [1]:
import os
import pandas

movies = pandas.read_json(os.path.join('data', 'movies_clean.jsonl.bz2'), lines=True).set_index('imdb_id')

In supervised machine learning, the goal is to make a prediction. In this case we will try to predict the rating that a movie will get, based in other available data.

Most machine learning models, like `logistic regression`, `random forest` and even `deep learning` models require that all the data is numerical, and that no missing data exists.

So, preprocessing is usually required in this sense. How this can be done? This depends of ourselves, about the available data, our business knowledge, and about how we master the *art* of machine learning.

### Feature extraction

Let us start with the title. What information can we obtain from it? Advanced techniques on natural language processing will be seen in the next section. But so far there are two "obvious" features that we can extract from it:
- The number of words
- The number of characters

Whether these are features that are relevant to estimate the rating of a movie we don't know. We can guess that they won't be the most relevant, but considering them should not harm (that may not be true, for a phenomena named the *curse of dimensionality*).

**Exercise:** We will keep the features we extract in a separate `DataFrame` named `data`. This is not necessary, but should make it easier to see which features are "ready". Add to this `DataFrame` the features with the number of words and characters in the title.

In [ ]:
data = movies[['num_ratings', 'rate']].copy()

Next feature to consider is `color`. In our data, the color can only be two values, `Color` or `Black and White`. So, even if the feature is a text, we can easily convert to a boolean value, on whether the value is one of them.

**Exercise:** Create a column `is_color` in `data`. In Python, `True == 1` and `False == 0`, so a boolean column is fine (or it can also be transformed to another numerical type like `int` or `float`).

The release date can also contain relevant information, even if as a date, it can't be directly processed. Some numerical information that can be extracted from a date:
- The days or years since the release of the movie (how old is it)
- The month of the release (are the best movies released in specific months?)
- The day of the week (probably not important, but who knows)

**Exercises:**
- Think what happened to the `release_date` that we extracted in the first section. Can you guess?
- Fix the problem. Can we use `pandas.to_datetime()` again? Is the `unit` parameter relevant?
- Extract the features mentioned above, and any other that you think can be relevant.

Finally, we will consider the genres. In this case we will use a technique named `one hot encoding` (aka `dummy variables`). It consists in creating a column for each possible value (e.g. `Romance`, `Action`, `Comedy`...) and set a value `0` or `1` depending on whether the row contains that value.

`pandas` contains two functions to extract the `dummy features`. The most commonly used  is `pandas.get_dummies` which converts a string `Series` into the dummies `DataFrame`. In case one row can have more than one value (like genres in our case) we can use `Series.str.get_dummies()`, which expects a string where the values are separated by a delimiter.

**Exercises:**
- Convert the content of `genres` from a `list` to a string separated by a delimiter of your choice. You can use the method `.apply()` to apply an arbitrary Python function to all the values of a `Series` at a time. 
- Get the dummy variables and add them to our `data.

Before we start using machine learning, we will split our dataset in two. The response feature we want to predict, and all the explanatory features.

**Exercise:** Create a `DataFrame` named `x` with all the columns except the rating, and a `Series` named `y` for the rating.

`scikit-learn` uses `numpy` arrays of `float` in most cases. We can usually provide the data in `pandas` containers of any type, and will do the conversions automatically. But let us do it explicitly.

**Exercise:** Convert `x` and `y` to `numpy` arrays of type `float`. You can get the `numpy` representation of a `pandas` object by calling its attribute `.values` (`pandas` internal representation uses `numpy`).

`scikit-learn` works with pipelines. Those pipelines are a set of transformations to the data and the final predictive model saved in a single object. The goal is for this object to remember required information to transform the data at the prediction time.

For example, if we want to scale the data (this is important for some models) we need to remember from the training to the prediction time, the `mean` used to scale the data. This is saved and kept in the pipeline.

To create a pipeline the simpler option is to use `sklearn.pipeline.make_pipeline` and provide each one of the steps.

**Exercise:** Create a pipeline with the next objects:
- `sklearn.preprocessing.Imputer`
- `sklearn.preprocessing.StandardScaler`
- `sklearn.linear_model.LinearRegression`

To train our model (make it learn how to predict based on our data), we use the method `.fit(x, y)`. Then, to make predictions, we use the method `.predict(x)`.

**Exercises:**
- Train the model built in the previous step, with the data we obtained.
- Make predictions for the same data.
- Convert the predictions to a `pandas.Series` and compare them with the actual values.

Can we trust the obtained results? Or in other words, if tomorrow a new movie is released, and we want to predict its rating, will our model will be performing in the same way?

The answer is that as we are predicting data that our model has already seen, it'd be as easy for the model as remembering every value, and then predicting the same for every row. That model will give perfect predictions in our evaluation, but it would most likely perform very poorly in new data. This phenomena is named *overfitting*.

As our model is as simple as a linear regression, it can not in practice overfit much. But the let us anyway get a good measure for the performance of the model.

**Exercise:** Let us start by creating two different datasets. We will use one to learn from the data, and the other to make predictions as if it was new data. `scikit-learn` provides the function `sklearn.model_selection.train_test_split` for it.

**Exercise:** Now we can simply repeat what we did before to train our model, and to make predictions, but we will use the new datasets. Obtain the predictions for the test dataset.

Finally, to compute the performance of the model, `scikit-learn` provides the most common used metrics for us. So, it is not necessary to compute them manually.

**Exercise:** Compute the performance of the model, with these two metrics:
- `sklearn.metrics.mean_absolute_error`
- `sklearn.metrics.r2_score`

Computing exact values is hard for machine learning. In most cases, machine learning is used to classify. This means that instead of trying to predict the exact rating of a movie (e.g. `8.25`), we will instead just try to classify them in two categories (e.g. `good` vs `bad` movies). To convert our problem from a regression to a classification problem, we need to change our `y`, so it contains `1` for good movies, and `0` for bad movies.

**Exercise:** Define a criterion on what is a good movie (a good idea can be movies with a rating higher than a given number). Then, convert the `y` variable in our data to a boolean column based on this criterion.

`scikit-learn` has a consistent API that makes it easy to transition from one problem to another, and still keep most of the same code.

To transition from a regression to a classification problem, these steps are required:
- Have a response variable that contains the categories (we just did that)
- Use a machine learning model that classifies (models are implemented different for classification and regression, so the same can not be used for both)
- In most cases we want to use `.predict_proba()` instead of `.predict()`, as it is more useful to know the probability of a movie being good, than a to just know that is good, without knowing how certain the model is.
- The measures to evaluate how good is the model are different too.

**Exercises:**
- Use the model `sklearn.ensemble.RandomForestClassifier` instead of the linear regression
- Train the model with the new data
- Predict the probability of every movie to be good
- Evaluate the performance of the model in the test set with `sklearn.metrics.accuracy_score`